<a href="https://colab.research.google.com/github/laurenx1/learning-systems/blob/main/final_regularized_linear_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# imports
import math
import numpy as np

# mounting google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# loading all the data
train = np.loadtxt('/content/drive/MyDrive/final_features.train')
test = np.loadtxt('/content/drive/MyDrive/final_features.test')


# splitting into digit intensity symmetry
digits = train[:, 0]
intensity = train[:, 1]
symmetry = train[:, 2]

# constructing x_train and y_train
x_train = np.column_stack((intensity, symmetry))
y_train = digits

digits2 = test[:, 0]
intensity2 = test[:, 1]
symmetry2 = test[:, 2]

# constructing x_test and y_test
x_test = np.column_stack((intensity2, symmetry2))
y_test = digits2

In [ ]:
# making our tranformed data, whether it be just adding a bias term or doing a non-linear transformation

# just adds the bias term of 1 to the front
def add_bias_term(X):
  x1, x2 = X
  return np.array([1, x1, x2])

# non-linear tranform as described in question 8
def feature_transform(X):
  x1, x2 = X
  return np.array([1, x1, x2, x1*x2, x1*x1, x2*x2])

# function that actually applies the tranform to all the data
def phi(X, transform_bool): # transform bool to determine which tranform is being done
  if transform_bool:
    Z = np.apply_along_axis(feature_transform, 1, X)
    return Z
  else:
    Z = np.apply_along_axis(add_bias_term, 1, X)
    return Z

In [ ]:
# calculating error, works for E_in or E_out, just switch out train or test data
def calc_error(Z, W, Y, N):
  pred = np.sign(np.dot(Z, W))
  E = sum(pred != Y) / N
  return E

In [ ]:
# transforming the data and performing least-squares linear regression on Z
def regularized_linear_regression(X, Y, lmbda):
  identity_matrix = np.identity(X.shape[1])
  W = np.linalg.inv(X.T @ X + lmbda * identity_matrix) @ X.T @ Y # the exact function given in the question
  return W # returns a regularized weight vector

In [ ]:
# implementing question 7

def q7(x_train, y_train, lmbda, transform_bool, classifiers):
  # Apply the transformation once before the loop
  z_train = phi(x_train, transform_bool)

  min_ein = float('inf')
  best_classifier = None

  for c in classifiers:
    # Filter the data for the selected digit
    y_digit_train = np.where(y_train == c, 1, -1)  # Set label +1 for the selected digit, -1 for others

    # No need to apply the transformation here, use the pre-transformed data
    w = regularized_linear_regression(z_train, y_digit_train, lmbda)
    ein = calc_error(z_train, w, y_digit_train, len(y_digit_train))

    print(f"Ein for classifier {c} versus all:", ein)

    if ein < min_ein:
      min_ein = ein
      best_classifier = c

  print(f"\nThe classifier with the lowest Ein is: {best_classifier} versus all, with Ein = {min_ein}")


# arguments as given to us in the problem
lambda_value = 1
transform_option = False  # would set to True for non-linear transform (like in question 8), False for bias term only
classifiers_to_compare = [5, 6, 7, 8, 9]

q7(x_train, y_train, lambda_value, transform_option, classifiers_to_compare)

Ein for classifier 5 versus all: 0.07625840076807022
Ein for classifier 6 versus all: 0.09107118365107666
Ein for classifier 7 versus all: 0.08846523110684405
Ein for classifier 8 versus all: 0.07433822520916199
Ein for classifier 9 versus all: 0.08832807570977919

The classifier with the lowest Ein is: 8 versus all, with Ein = 0.07433822520916199


Ein for classifier 5 versus all: 0.07625840076807022

Ein for classifier 6 versus all: 0.09107118365107666

Ein for classifier 7 versus all: 0.08846523110684405

Ein for classifier 8 versus all: 0.07433822520916199

Ein for classifier 9 versus all: 0.08832807570977919

The classifier with the lowest Ein is: 8 versus all, with Ein = 0.07433822520916199

In [ ]:
# implementing question 8
def q8(x_train, y_train, x_test, y_test, lmbda, transform_bool, classifiers):
  z_train = phi(x_train, transform_bool)
  z_test = phi(x_test, transform_bool)

  min_eout = float('inf')
  best_classifier_1 = None

  for c in classifiers:
    y_digit_train = np.where(y_train == c, 1, -1)
    y_digit_test = np.where(y_test == c, 1, -1)

    w = regularized_linear_regression(z_train, y_digit_train, lmbda)
    eout = calc_error(z_test, w, y_digit_test, len(y_digit_test))

    print(f"Eout for classifier {c} versus all:", eout)

    if eout < min_eout:
      min_eout = eout
      best_classifier_1 = c

  print(f"\nThe classifier with the lowest Eout is: {best_classifier_1} versus all, with Eout = {min_eout}")

# args for 8
lambda_value = 1
to_transform = True
classifiers_array = [0, 1, 2, 3, 4]

q8(x_train, y_train, x_test, y_test, lambda_value, to_transform, classifiers_array)

Eout for classifier 0 versus all: 0.10662680617837568
Eout for classifier 1 versus all: 0.02192326856003986
Eout for classifier 2 versus all: 0.09865470852017937
Eout for classifier 3 versus all: 0.08271051320378675
Eout for classifier 4 versus all: 0.09965122072745392

The classifier with the lowest Eout is: 1 versus all, with Eout = 0.02192326856003986


Eout for classifier 0 versus all: 0.10662680617837568

Eout for classifier 1 versus all: 0.02192326856003986

Eout for classifier 2 versus all: 0.09865470852017937

Eout for classifier 3 versus all: 0.08271051320378675

Eout for classifier 4 versus all: 0.09965122072745392

The classifier with the lowest Eout is: 1 versus all, with Eout = 0.02192326856003986

In [ ]:
def q9(x_train, y_train, x_test, y_test, lmbda, classifiers):
  q8(x_train, y_train, x_test,y_test, lmbda, False, classifiers)
  q8(x_train, y_train, x_test, y_test, lmbda, True, classifiers)

classifiers_9 = [0,1,2,3,4,5,6,7,8,9]
l = 1
q9(x_train, y_train, x_test, y_test, 1, classifiers_9)

Eout for classifier 0 versus all: 0.11509715994020926
Eout for classifier 1 versus all: 0.02242152466367713
Eout for classifier 2 versus all: 0.09865470852017937
Eout for classifier 3 versus all: 0.08271051320378675
Eout for classifier 4 versus all: 0.09965122072745392
Eout for classifier 5 versus all: 0.07972097658196313
Eout for classifier 6 versus all: 0.08470353761833582
Eout for classifier 7 versus all: 0.07324364723467862
Eout for classifier 8 versus all: 0.08271051320378675
Eout for classifier 9 versus all: 0.08819133034379671

The classifier with the lowest Eout is: 1 versus all, with Eout = 0.02242152466367713
Eout for classifier 0 versus all: 0.10662680617837568
Eout for classifier 1 versus all: 0.02192326856003986
Eout for classifier 2 versus all: 0.09865470852017937
Eout for classifier 3 versus all: 0.08271051320378675
Eout for classifier 4 versus all: 0.09965122072745392
Eout for classifier 5 versus all: 0.07922272047832586
Eout for classifier 6 versus all: 0.0847035376183

Ein for classifier 0 versus all: 0.10931285146070498

Ein for classifier 1 versus all: 0.01522424907420107

Ein for classifier 2 versus all: 0.10026059525442327

Ein for classifier 3 versus all: 0.09024825126868742

Ein for classifier 4 versus all: 0.08942531888629818

Ein for classifier 5 versus all: 0.07625840076807022

Ein for classifier 6 versus all: 0.09107118365107666

Ein for classifier 7 versus all: 0.08846523110684405

Ein for classifier 8 versus all: 0.07433822520916199

Ein for classifier 9 versus all: 0.08832807570977919

The classifier with the lowest Ein is: 1 versus all, with Ein = 0.01522424907420107

Eout for classifier 0 versus all: 0.10662680617837568

Eout for classifier 1 versus all: 0.02192326856003986

Eout for classifier 2 versus all: 0.09865470852017937

Eout for classifier 3 versus all: 0.08271051320378675

Eout for classifier 4 versus all: 0.09965122072745392

Eout for classifier 5 versus all: 0.07922272047832586

Eout for classifier 6 versus all: 0.08470353761833582

Eout for classifier 7 versus all: 0.07324364723467862

Eout for classifier 8 versus all: 0.08271051320378675

Eout for classifier 9 versus all: 0.08819133034379671

The classifier with the lowest Eout is: 1 versus all, with Eout = 0.02192326856003986


In [ ]:
def q10(x_train, y_train, x_test, y_test, lmbda_arr, transform_bool, class_pairs):
    # Apply the transformation once before the loop for both training and test data
  z_train = phi(x_train, transform_bool)
  z_test = phi(x_test, transform_bool)

  min_ein = float('inf')
  min_eout = float('inf')
  best_classifier = None
  best_lambda = None

  for lmbda in lmbda_arr:
    for pair in class_pairs:
      digit1, digit2 = pair

      # Filter the training data for the selected digit pair
      indices_train = np.where((y_train == digit1) | (y_train == digit2))
      x_digit_train = x_train[indices_train]
      y_digit_train = np.where(y_train[indices_train] == digit1, 1, -1)

      # Filter the test data for the selected digit pair
      indices_test = np.where((y_test == digit1) | (y_test == digit2))
      x_digit_test = x_test[indices_test]
      y_digit_test = np.where(y_test[indices_test] == digit1, 1, -1)

      # No need to apply the transformation here, use the pre-transformed data
      w = regularized_linear_regression(z_train[indices_train], y_digit_train, lmbda)

      ein = calc_error(z_train[indices_train], w, y_digit_train, len(y_digit_train))
      eout = calc_error(z_test[indices_test], w, y_digit_test, len(y_digit_test))

      print(f"Ein for classifier {digit1} versus {digit2} with lambda = {lmbda}: {ein}")
      print(f"Eout for classifier {digit1} versus {digit2} with lambda = {lmbda}: {eout}")

      if ein < min_ein:
        min_ein = ein
        best_classifier = pair
        best_lambda = lmbda

      if eout < min_eout:
        min_eout = eout

  #print(f"\nThe best classifier with the lowest Ein is: {best_classifier} with lambda = {best_lambda} and Ein = {min_ein}")
  #print(f"The best classifier with the lowest Eout is: {best_classifier} with lambda = {best_lambda} and Eout = {min_eout}")



lmbda_arr = [0.01, 1]
classifier_pairs = [(1, 5)]
transform = True
q10(x_train, y_train, x_test, y_test, lmbda_arr, transform, classifier_pairs)


Ein for classifier 1 versus 5 with lambda = 0.01: 0.004484304932735426

Eout for classifier 1 versus 5 with lambda = 0.01: 0.02830188679245283

Ein for classifier 1 versus 5 with lambda = 1: 0.005124919923126201

Eout for classifier 1 versus 5 with lambda = 1: 0.025943396226415096